In [1]:
import os
import sys
import numpy as np 
import pandas as pd
from importlib import reload
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
sys.path.insert(0,'../embeddings')
import utilities as ut
import quora_vocab as qv
import svd_embeddings as svd_em

In [3]:
DATA_PATH = '~/google_drive/data/quora/'
DATA_FILE = '{}{}'.format(DATA_PATH,'train.csv')

### Data Loading

In [4]:
data = pd.read_csv(DATA_FILE)
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


### Tokenizing Each Comment

In [5]:
data_pos_tokenized = [[vec[0],ut.canon_token_sentence(vec[1]),vec[2]] 
                      for vec in data.to_numpy() if vec[2] == 1]  
data_neg_tokenized = [[vec[0],ut.canon_token_sentence(vec[1]),vec[2]] 
                      for vec in data.to_numpy() if vec[2] == 0]   

### Tokenized Innapropriate Question Example

In [6]:
' '.join(data_pos_tokenized[103][1])

'what do democrats think of the fact that i am homophobic and was born this way will this make you rethink your rhetoric'

#### Tokenized Appropriate Question Example

In [7]:
' '.join(data_neg_tokenized[11][1])

'how were the calgary flames founded'

### Train Test Split

In [8]:
NVALID = 5000
NTRAIN = 150000
from random import shuffle
train_data = data_pos_tokenized[NVALID:NTRAIN+NVALID] + data_neg_tokenized[NVALID:NTRAIN+NVALID]
shuffle(train_data)
valid_data = data_pos_tokenized[:NVALID] + data_neg_tokenized[:NVALID]
shuffle(valid_data)

### Class Construction

In [25]:
reload(qv)
comments = qv.CommentVocab(data_pos_tokenized+data_neg_tokenized)

Processing Comments: 100%|██████████| 1306122/1306122 [00:37<00:00, 34390.02it/s]


In [27]:
len(comments.unigram_counts[1].keys())+len(comments.unigram_counts[0].keys())

231355

In [30]:
comments.unigram_totals

[15448145, 1408137]

### Comparison of Question Length by Class

In [31]:
pos_lengths = [ len(vec[1]) for vec in data_pos_tokenized]
neg_lengths = [ len(vec[1]) for vec in data_neg_tokenized]
print('{}\n{}\n'.format('----Length of Positive Examples-----',pd.Series(pos_lengths).describe()))
print('{}\n{}'.format('----Length of Negative Examples-----',pd.Series(neg_lengths).describe()))

----Length of Positive Examples-----
count    80810.000000
mean        17.425282
std          9.641367
min          1.000000
25%         10.000000
50%         15.000000
75%         23.000000
max         83.000000
dtype: float64

----Length of Negative Examples-----
count    1.225312e+06
mean     1.260752e+01
std      6.811622e+00
min      2.000000e+00
25%      8.000000e+00
50%      1.100000e+01
75%      1.500000e+01
max      1.320000e+02
dtype: float64


In [32]:
comments.comment_length_graph()

#### Comments

* We can see from the summary statistics and the empirical distribution that the distribution of inappropriate questions has far fatter tails than the distribution of appropriate questions demonstrating that innaproriate questions tend to be somewhat longer. This matches what one would expect from the description of the innapropriate class which includes questions that are really statements of some position that the user has, or just generalized trolling.

* On the other hand one can interpret the relative shortness of appropriate question as reflecting more concise and clearly stated questions. 

### Comparison of Word Frequency By Class

In [33]:
comments.word_frequency_graphs(min_rank=3,max_rank=50)

In [34]:
comments.word_count_difference_graph(gram_length=1,num=20)

#### Comments

* The preceeding graphs show the differneces in word frequency between classes.

* One thing that the last graph shows, which one should expect, is the higher frequency ,in the inappropriate class, of words that that are either polarizing e.g. 'trump' or are groups who are often subject to claims of supremacy or inferiority depending on the prejudices of the asker e.g. 'white', 'men', 'women', 'muslims'.

* One interesting unexpected result is the difference in the frequency of different interogative words used in the different classes of questions e.g. 'what', 'how', and 'which' are more frequent in the appropriate class, while interogative word 'why' is far more likely in the innappropriate class. 

* Perhaps the increased use of 'why' in the innapropriate class is due to the ease at which one can disguise a statement of a dubious, non-factual nature as a why question. For example the question 'why are aliens manging my local dairy queen?' presents the premise, that aliens are managing a dairy queen somewhere, as fact and implicitly requires the reader to accept the premise in order to respond directly to it. Note that questions like, 'which of my local dairy queens is managed by aliens', 'how can my local dairy queen be managed by aliens', and 'does/can my local dairy queen be managed by aliens' do not require the answerer to accept the premise in order to respond. 

In [35]:
comments.word_frequency_graphs(gram_length=2,min_rank=3,max_rank=50)

In [36]:
comments.word_count_difference_graph(gram_length=3,num=20)

#### Comments

* Here when we examine bigrams and find the same trends as we found in the analysis of unigrams 

* In the inappropriate example we find a comparitively high frequency of 'why' interogative words and phrases which can be formulated as a statement of dubious facts, that implicitly require the answer to assume this premise in order to answer it. 

In [45]:
cmnts = [val[1] for val in data_pos_tokenized+data_neg_tokenized]
test = svd_em.SVD_Embeddings(cmnts,tokenized=True)

Conversion to ID: 100%|██████████| 1306122/1306122 [00:10<00:00, 125866.15it/s]


In [ ]:
import embedding_utilities as embd_util

In [97]:
reload(embd_util)
filename = '../embeddings/quora_full_embeds.txt'
embeddings, token_to_row_number = embd_util.embedding_extraction(filename)
id_sequences = embd_util.token_sequence_to_id_sequence(cmnts,token_to_row_number,'UNK')

In [98]:
sum([len(seq) for seq in id_sequences])/len(id_sequences)

100.0

In [94]:
[len(seq) for seq in id_sequences]

[111,
 114,
 113,
 131,
 102,
 101,
 110,
 119,
 108,
 118,
 132,
 111,
 103,
 111,
 106,
 102,
 99,
 102,
 103,
 105,
 90,
 88,
 117,
 84,
 89,
 84,
 88,
 115,
 92,
 91,
 77,
 85,
 70,
 95,
 82,
 99,
 77,
 84,
 75,
 71,
 88,
 79,
 75,
 64,
 74,
 72,
 64,
 66,
 67,
 65,
 85,
 67,
 62,
 60,
 58,
 65,
 66,
 83,
 57,
 60,
 64,
 49,
 60,
 76,
 43,
 60,
 59,
 42,
 41,
 51,
 45,
 50,
 39,
 50,
 45,
 33,
 56,
 30,
 27,
 30,
 30,
 70,
 52,
 32,
 30,
 23,
 22,
 32,
 25,
 41,
 18,
 40,
 51,
 27,
 34,
 20,
 19,
 12,
 26,
 6,
 20,
 9,
 13,
 23,
 28,
 24,
 14,
 10,
 12,
 20,
 7,
 34,
 8,
 8,
 34,
 39,
 27,
 23,
 14,
 12,
 8,
 15,
 9,
 23,
 12,
 26,
 14,
 10,
 11,
 13,
 18,
 8,
 13,
 16,
 11,
 5,
 20,
 11,
 24,
 15,
 9,
 21,
 11,
 17,
 14,
 15,
 38,
 10,
 6,
 9,
 16,
 13,
 13,
 10,
 14,
 14,
 24,
 21,
 13,
 12,
 23,
 17,
 9,
 15,
 14,
 8,
 9,
 11,
 26,
 9,
 31,
 9,
 11,
 7,
 12,
 24,
 11,
 31,
 9,
 19,
 24,
 21,
 29,
 8,
 27,
 15,
 29,
 6,
 40,
 9,
 15,
 16,
 39,
 35,
 8,
 10,
 6,
 16,
 15,
 7,
 15,